# Intermediate Joins in SQL

## Working With Larger Databases

In the previous mission we learned how to use joins to combine data from two tables within a database. In reality, most databases have more than 2 tables, so we'll need strategies to be able to write queries to combine data from 3 or more tables.<br>

In this mission we'll learn some new techniques to work with the sort of databases that most businesses will use. We'll be working with a modified version of a database called Chinook. The [Chinook](https://github.com/lerocha/chinook-database) database contains information about a fictional digital music shop - kind of like a mini-iTunes store.<br>

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Like with our Factbook database, a schema diagram will help us understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

![https://s3.amazonaws.com/dq-content/189/chinook-schema.svg](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

Looking at all those tables can be overwhelming at first, but generally speaking we will only need to think about the specific tables that have the data we require and their connections. Before we look at some specific tables, let's take a moment to understand the different parts of the schema diagram.
* Tables names are shown in bold, with the columns in each table listed below.
* Each table has one or more columns with shading, which indicates those columns are a primary key. We'll learn more about primary keys in a later mission, but for now all you need to know is that each row's primary key must be unique.
* Relationships between tables are shown using lines between the tables. The lines indicate which columns are related. You may notice that at least one 'end' of the relationship will be a primary key.

As you work through this mission, you may need to refer back to the schema diagram. This is normal, so don't be bothered when that happens. It can also be helpful to write a simple query to check the column names and some example data from any of the tables, for instance:

```sql
SELECT * FROM album LIMIT 3;
```

album_id|title|artist_id
---|---|---
1|For Those About To Rock We Salute You|1
2|Balls to the Wall|2
3|Restless and Wild|2

In [5]:
import sqlite3
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline
conn = sqlite3.connect("data/chinook.db")

## Joining Three Tables

Our first task is to gather some information on a specific purchase. For one single purchase (`invoice_id`) we want to know, for each track purchased:
* The id of the track.
* The name of the track.
* The name of media type of the track.
* The price that the customer paid for the track.
* The quantity of the track that was purchased.

To gather this information, we will need to write a query that joins 3 tables: `invoice_line`, `track`, and `media_type`. To make this easier, a schema diagram of just these three tables is below:

![https://s3.amazonaws.com/dq-content/189/schema_il_t_mt.svg](https://s3.amazonaws.com/dq-content/189/schema_il_t_mt.svg)

If you didn't explore these 3 tables in the previous screen, take a moment now to explore them:
* For each of `invoice_line`, `track`, and `media_type`:
  * Write a simple query, using either the console or code box, to view the first few rows of each of these tables.
  * Look at the result of the query to become familiar with the columns and what sort of data we are working with.

Now that you're familiar with the data, we can look at the syntax for joining data from more than 2 tables.

```sql
SELECT [column_names] FROM [table_name_one]
[join_type] JOIN [table_name_two] ON [join_constraint]
[join_type] JOIN [table_name_three] ON [join_constraint];
```

Joining multiple tables is as simple as adding an extra `JOIN` clause. The SQL engine interprets joins in order, so the first join will be executed, and then the second join will be executed against the result of the first join. Because of this, we can first build our query in steps:
* with 0 joins.
* with 1 join.
* with 2 joins.

We will use the `invoice_line` table in our `FROM` clause, since it contains 3 of the 5 columns we want in our final query. You don't have to follow this pattern but if you're not sure, it's a good place to start. Since our tasks involves looking for information about a specific `invoice_id`, let's choose an `invoice_id` value of `3`. Selecting all lines from `invoice_line` with an `invoice_id` is straightforward:

```sql
SELECT * FROM invoice_line
WHERE invoice_id = 3;
```


In [6]:
q = "SELECT * FROM invoice_line LIMIT 5;"
pd.read_sql_query(q, conn)

,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1


In [7]:
q = "SELECT * FROM invoice_line where invoice_id=3;"
pd.read_sql_query(q, conn)

,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,27,3,2516,0.99,1
1,28,3,2646,0.99,1


Now we can use an inner join to add the data from the track table.

```sql
SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
WHERE invoice_id = 3;
```

In [9]:
q = '''
        select * from invoice_line il
        inner join track t on t.track_id = il.track_id
        where invoice_id = 3;
'''
pd.read_sql(q, conn)

,invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99
1,28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Ji...",142080,4663988,0.99


![https://s3.amazonaws.com/dq-content/189/multiple_joins_step1.svg](https://s3.amazonaws.com/dq-content/189/multiple_joins_step1.svg)

Next, we add our second join to add the data from the media_type table.

```sql
SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE invoice_id = 3;
```

In [11]:
q = '''
        select * from invoice_line li
        inner join track t on t.track_id = li.track_id
        inner join media_type mt on mt.media_type_id = t.media_type_id
        where invoice_id = 3;
'''
pd.read_sql(q, conn)

,invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price,media_type_id,name
0,27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99,1,MPEG audio file
1,28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Ji...",142080,4663988,0.99,1,MPEG audio file


![https://s3.amazonaws.com/dq-content/189/multiple_joins_step2.svg](https://s3.amazonaws.com/dq-content/189/multiple_joins_step2.svg)

The last step is to alter the `SELECT` clause to include only the columns we require - let's do that now with a different order.

Write a query that gathers data about the invoice with an `invoice_id` of 4. Include the following columns in order:
* The id of the track, `track_id`.
* The name of the track, `track_name`.
* The name of media type of the track, `track_type`.
* The price that the customer paid for the track, `unit_price`.
* The quantity of the track that was purchased, `quantity`.

In [20]:
query = '''
        select t.track_id track_id,
                t.name track_name,
                mt.name track_type,
                li.unit_price unit_price,
                li.quantity quantity
        from invoice_line li
        inner join track t on t.track_id = li.track_id
        inner join media_type mt on mt.media_type_id = t.media_type_id
        where invoice_id = 4;
'''
pd.read_sql(query, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


## Joining More Than Three Tables

Let's extend the query we wrote in the previous screen by adding the artist for each track. If you examine the schema, you'll notice that the data for the artist's name is not directly connected to the `track` table.<br>

![Schema of track, album, and artist](https://s3.amazonaws.com/dq-content/189/schema_t_al_ar.svg)

In this case, we will need to join two new tables to our existing query:
* `artist`, which contains the artist name data that we need
* `album`, which has a column common to each of the `artist` and `track` tables which allows us to join those two tables.

Our select clause won't actually use any of the columns from the `album` table. This is quite common when writing more complex queries to need to join a specific table because it will let you join to another table.<br>

The query from the previous screen is included here:

```sql
SELECT
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE il.invoice_id = 4;
```

Add a column containing the artists name to the query from the previous screen.
* The column should be called `artist_name`
* The column should be placed between `track_name` and `track_type`

In [21]:
query = '''
        select t.track_id track_id,
                t.name track_name,
                ar.name artist_name,
                mt.name track_type,
                li.unit_price unit_price,
                li.quantity quantity
        from invoice_line li
        inner join track t on t.track_id = li.track_id
        inner join media_type mt on mt.media_type_id = t.media_type_id
        inner join album al on t.album_id = al.album_id
        inner join artist ar on ar.artist_id = al.artist_id
        where invoice_id = 4;
'''
pd.read_sql(query, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


## Combining Multiple Joins with Subqueries

Because the `invoice_line` table contains each individual song from each customer purchase, it contains information about which songs are purchased the most. We can use the table to find out which artists are purchased the most. Specifically, what we want to produce is a *query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased*.<br>

We'll need to use a `GROUP BY` clause to get the number of tracks purchased from each artist, but before we do we'll have to join a few tables. As we start thinking about how we will write this query, we can look at the database schema from the first screen of this mission and think about what tables we will need to include.<br>

Writing our query would be a lot easier if we had one table that contained both the `track.track_id` and the `artist.name` columns. We can write a subquery that creates this table for us, and then use that subquery to calculate our answer. This means our process will be:
* Write a subquery that produces a table with track.track_id and artist.name,
* Join that subquery to the invoice_line table,
* Use a GROUP BY statement to calculate the number of times each artist has had a track purchased, and find the top 10.

We can write our subquery by joining `album` to `track` and then `artist` to `album`, just like we did in the previous screen. We'll add an `ORDER BY` and `LIMIT` to our query so we're only looking at manageable sample of the data, but we'll remove it when we move to the next step.

In [22]:
q = '''
    SELECT
        t.track_id,
        ar.name artist_name
    FROM track t
    INNER JOIN album al ON al.album_id = t.album_id
    INNER JOIN artist ar ON ar.artist_id = al.artist_id
    ORDER BY 1 LIMIT 10;
'''
pd.read_sql(q, conn)

,track_id,artist_name
0,1,AC/DC
1,2,Accept
2,3,Accept
3,4,Accept
4,5,Accept
5,6,AC/DC
6,7,AC/DC
7,8,AC/DC
8,9,AC/DC
9,10,AC/DC


Next, we need to join this subquery to our invoice_line table. We'll give our subquery an alias ta for 'track artist' to make it easier to refer to. Again, we'll add an ORDER BY and LIMIT statement to this step so our output is more manageable.

In [23]:
q = '''
    SELECT
        il.invoice_line_id,
        il.track_id,
        ta.artist_name
    FROM invoice_line il
    INNER JOIN (
                SELECT
                    t.track_id,
                    ar.name artist_name
                FROM track t
                INNER JOIN album al ON al.album_id = t.album_id
                INNER JOIN artist ar ON ar.artist_id = al.artist_id
               ) ta
               ON ta.track_id = il.track_id
    ORDER BY 1 LIMIT 5;
'''
pd.read_sql(q, conn)

,invoice_line_id,track_id,artist_name
0,1,1158,Guns N' Roses
1,2,1159,Guns N' Roses
2,3,1160,Guns N' Roses
3,4,1161,Guns N' Roses
4,5,1162,Guns N' Roses


At first it might look like we've done something wrong, because the artist for all rows is Guns N' Roses, but that's because the very first order in our table is a customer who purchased an entire Guns N' Roses album! All that remains now is for us to add our `GROUP BY` clause, remove the extra columns and use `ORDER BY` and `LIMIT` clauses to select the 10 most popular artists.

In [24]:
q = '''
    select ta.artist_name artist,
            count(*) track_purchased
    from invoice_line il
    inner join (
            select t.track_id,
                    ar.name artist_name
            from track t
            inner join album al
            on al.album_id = t.album_id
            inner join artist ar
            on ar.artist_id = al.artist_id
    ) ta
    on ta.track_id = il.track_id
    group by 1
    order by 2 desc
    limit 10;
'''

pd.read_sql(q, conn)

,artist,track_purchased
0,Queen,192
1,Jimi Hendrix,187
2,Nirvana,130
3,Red Hot Chili Peppers,130
4,Pearl Jam,129
5,AC/DC,124
6,Guns N' Roses,124
7,Foo Fighters,121
8,The Rolling Stones,117
9,Metallica,106


* Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should return the following columns, in order:
  * `album`, the title of the album
  * `artist`, the artist who produced the album
  * `tracks_purchased` the total number of tracks purchased from that album
* Your query should list the albums from most tracks purchased to least tracks purchased.

In [ ]:
select ta.album album,
                ta.artist artist,
                count(*) track_purchased
    from invoice_line il
    inner join (
            select t.track_id,
                    ar.name artist,
                    al.title album
            from track t
            inner join album al
            on al.album_id = t.album_id
            inner join artist ar
            on ar.artist_id = al.artist_id
            ) ta
    on ta.track_id = il.track_id
    group by 1, 2
    order by 3 desc
    limit 5;

In [30]:
q = '''
    select ta.album album,
                ta.artist artist,
                count(*) track_purchased
    from invoice_line il
    inner join (
            select t.track_id,
                    ar.name artist,
                    al.title album
            from track t
            inner join album al
            on al.album_id = t.album_id
            inner join artist ar
            on ar.artist_id = al.artist_id
            ) ta
    on ta.track_id = il.track_id
    group by 1, 2
    order by 3 desc
    limit 5;
'''

pd.read_sql(q, conn)

,album,artist,track_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


## Recursive Joins

In some cases, there can be a relation between two columns within the same table. We can see that in our `employee` table, where there is a `reports_to` column that has a relation to the `employee_id` column within the same table. You might like to explore the `employee` table using some simple queries in the console.<br>

The `reports_to` column identifies each employee's supervisor. If we wanted to create a report of each employee and their supervisor's name, we would need some way of joining a table to itself. Doing this is called a **recursive join**.<br>

Technically, a recursive join will use one of the other standard joins— usually an `INNER JOIN` or `LEFT JOIN` since these are the most commonly used joins— and aliases to distinguish between the table on each side of the join. Here's a simple example of a recursive join in action:

In [35]:
q = '''
    select * from employee limit 5;
'''
pd.read_sql(q, conn)

,employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [37]:
q = '''
SELECT e1.employee_id,
        e1.reports_to,
        e2.employee_id,
        e2.reports_to
FROM employee e1
INNER JOIN employee e2 
on e1.reports_to = e2.employee_id
LIMIT 4;
'''
pd.read_sql(q, conn)

# This query produces the following table:

,employee_id,reports_to,employee_id,reports_to
0,2,1,1,NaN
1,6,1,1,NaN
2,3,2,2,1.0
3,4,2,2,1.0


In [33]:
q = '''
SELECT
    e1.employee_id,
    e2.employee_id supervisor_id
FROM employee e1
INNER JOIN employee e2 
on e1.reports_to = e2.employee_id
LIMIT 4;
'''
pd.read_sql(q, conn)

# This query produces the following table:

,employee_id,supervisor_id
0,2,1
1,6,1
2,3,2
3,4,2


To make our report more meaningful, we'll need to add some extra columns. One thing that would be nice is being able to combine the `first_name` and `last_name` columns into a single column. We can do that using the **concatenate operator**: `||`.<br>

You can explore how the concatenate operator works using the console:

```
>>> SELECT ("this" || "is" || "my" || "string");

    thisismystring
```

Here's a few different examples of using concatenation, this time with columns from the album table:

In [34]:
q = '''
SELECT
    album_id,
    artist_id,
    "album id is" || album_id col_1,
    "artist id is" || artist_id col2,
    album_id || artist_id col3
FROM album LIMIT 3;
'''

pd.read_sql(q, conn)

,album_id,artist_id,col_1,col2,col3
0,1,1,album id is1,artist id is1,11
1,4,1,album id is4,artist id is1,41
2,2,2,album id is2,artist id is2,22


You'll notice that the sql engine will concatenate multiple columns, columns with a string, and that the sql engine handles converting different types where needed.<br>

Let's use what we've learned about recursive joins and the concatenation operator to create a report about employees and their supervisors.

Write a query that returns a information about each employer and their supervisor. Your query should return the following columns, in order:
* `employee_name` - containing the `first_name` and `last_name` columns separated by a space, eg `Luke Skywalker`
* `employee_title` - the title of that employee
* `supervisor_name` - the first and last name of the person the employee reports to, in the same format as `employee_name`
* `supervisor_title` - the title of the person the employee reports to

The report should include employees even if they do not report to another employee.

In [41]:
q = '''
SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1
left join employee e2
on e1.reports_to = e2.employee_id
'''

pd.read_sql(q, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


## Pattern Matching Using Like

Let's say you're working in the as a sales support agent for Chinook, and you get back from lunch and see that one of your colleagues has left a phone message on your desk: "Call Jen". Writing a query to get this information out of the database might seem simple at first:

In [42]:
q = '''
    select
        first_name,
        last_name,
        phone
    from customer
    where first_name = "Jen";
'''
pd.read_sql(q, conn)

,first_name,last_name,phone


No rows were returned by your query! Suddenly you realize— **the name in the database might not be exactly "Jen", it might be Jenny, Jennifer, Jenna, or something else**. Rather than having to keep trying different combinations, we can use the `LIKE` operator to find pattern matches. The syntax for `LIKE` is as follows:

```sql
WHERE [column_name] LIKE "[pattern]"
```

Your pattern should be the substring you want to match for, and one or more % characters:

* `%Jen` - will match `Jen` at the end of a string, eg `Sarah-Jen`
* `Jen%` - will match `Jen` at the start of a string, eg `Jenny`
* %Jen% - will match Jen anywhere within the string, eg `Kris Jenner`

We'll use the last pattern to give ourselves the best chance of a match:

In [49]:
q = '''
    select
        first_name,
        last_name,
        phone
    from customer
    where first_name like "%Jen%"
'''
pd.read_sql(q, conn)

,first_name,last_name,phone
0,Jennifer,Peterson,+1 (604) 688-2255


Perfect - we have found one match and have Jennifer's number so we can return her call.<br>

Keep in mind that in SQLite `LIKE` is **case insensitive**, so `LIKE "%jen%"` will match `Jen` and `JEN` and `JeN`. Other flavors of SQL may be case sensitive, so you may need to use the [LOWER() function](https://stackoverflow.com/questions/18853452/sql-select-like-insensitive-casing) to get a case insensitive match.<br>

Let's use LIKE to find a different customer from our database.

* You have just returned from lunch to see another phone message on your desk: "Call Belle". Write a query that finds the contact details a customer with a `first_name` containing `Belle` from the database.
* Your query should include the following columns, in order:
  * `first_name`
  * `last_name`
  * `phone`

In [50]:
query = '''
        select first_name,
                last_name,
                phone
        from customer
        where first_name like "%Belle%"
'''

pd.read_sql(query, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


## Generating Columns With the Case Statement

Sometimes, you'll want to add a some sort of categorization to your query. Let's say we wanted to generate a report summarizing each customer purchases. It might be nice to create a column that puts each customer into a category:

* `small spender` - If the customer's total purchases are less than $40.
* `big spender` - If the customer's total purchases are greater than $100.
* `regular` - If the customer's total purchases are between $40 and $100 (inclusive).

To achieve this, we'll need to use the case statement. The **case statement** acts like a series of if/then options for a new column. The syntax for `CASE` is:

```sql
CASE
    WHEN [comparison_1] THEN [value_1]
    WHEN [comparison_2] THEN [value_2]
    ELSE [value_3]
    END
    AS [new_column_name]
```

There can 1 or more `WHEN` lines, and the `ELSE` line is optional— without it, rows that don't match any `WHEN` will be assigned a null value. Let's start by looking at a simple example. The `media_type` table contains the name of five different media types:

In [51]:
q = '''
    select * from media_type;
'''
pd.read_sql(q, conn)

,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


Let's look at how we can use CASE to add a new column protected, which indicates whether each media type is protected.

In [52]:
q = '''
    select
        media_type_id,
        name,
        CASE
            WHEN name LIKE '%Protected%' THEN 1
                                        ELSE 0
                                        END
                                        AS protected
    from media_type;
'''
pd.read_sql(q, conn)

,media_type_id,name,protected
0,1,MPEG audio file,0
1,2,Protected AAC audio file,1
2,3,Protected MPEG-4 video file,1
3,4,Purchased AAC audio file,0
4,5,AAC audio file,0


In this example, our `CASE` statement has a single `WHEN` which looks for a partial match of the string `Protected` in the `name` column. Any rows with a match get a value of `1`, all other rows get `0`.<br>

Let's now put `CASE` into practice using our initial scenario, where we want to categorize customers by their purchase history.<br>

You'll need to know that you can't use aliases in the `WHEN` line, so when writing a `CASE` statement that deals with aggregate functions, you will need to include the aggregate function in each `WHEN` line.

* Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.
* Your query should include the following columns, in order:
  * `customer_name` - containing the `first_name` and `last_name` columns separated by a space, eg `Luke Skywalker`.
  * `number_of_purchases`, counting the number of purchases made by each customer.
  * `total_spent` - the sum of customers total purchases made by each customer.
  * `customer_category` - a column that categorizes the customer based on their total purchases. The column should contain the following values:
    * `small spender` - If the customer's total purchases are less than \$40.
    * `big spender` - If the customer's total purchases are greater than \$100.
    * `regular` - If the customer's total purchases are between \$40 and \$100 (inclusive).
* Order your results by the `customer_name` column.

In [53]:
# show the head of customer table

q_customer_head = 'select * from customer limit 5;'
pd.read_sql(q_customer_head, conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [55]:
# explore 'invoice'+'invoice_line' table to check the 'total' column
# means total payments or not.

q = '''
    select * 
    from invoice iv
    inner join invoice_line ivl
    on iv.invoice_id = ivl.invoice_line_id
    limit 5;
'''
pd.read_sql(q, conn)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,1,1,1158,0.99,1
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90,2,1,1159,0.99,1
2,3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98,3,1,1160,0.99,1
3,4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92,4,1,1161,0.99,1
4,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83,5,1,1162,0.99,1


In [59]:
# 'total' column in 'invoice' table 
# DOES IT STAND FOR 'accumulative money spent'?

q = '''
    select *
    from invoice iv
    inner join invoice_line ivl
    on iv.invoice_id = ivl.invoice_line_id
    group by 2
    order by 2
    limit 10;
'''
pd.read_sql(q, conn)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,544,1,2020-07-24 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,8.91,544,69,86,0.99,1
1,592,2,2020-11-21 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,7.92,592,73,2405,0.99,1
2,517,3,2020-05-16 00:00:00,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,8.91,517,66,1785,0.99,1
3,474,4,2020-02-04 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,6.93,474,60,2416,0.99,1
4,588,5,2020-11-12 00:00:00,Klanova 9/506,Prague,None,Czech Republic,14700,0.99,588,73,2401,0.99,1
5,579,6,2020-10-23 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,10.89,579,73,2392,0.99,1
6,553,7,2020-08-26 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,4.95,553,69,95,0.99,1
7,431,8,2019-09-21 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,9.90,431,55,2552,0.99,1
8,470,9,2020-01-29 00:00:00,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,1.98,470,60,479,0.99,1
9,534,10,2020-06-25 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,5.94,534,68,2265,0.99,1


### Note.
* 'total' column in 'invoice' table - accumulative money spent by each customer
* 'customer_id' is unique and does not appear more than once in 'invoice' table
* we can take 'total' in 'invoice' table as the total money spent by each customer.


In [67]:
# to get the number of purchasement by each customer,
# group by 'invoice_id'.

query = '''
    select 
        *,
        count(*) as num_purchases
    from customer c
    inner join invoice iv on c.customer_id = iv.customer_id
    inner join invoice_line ivl on iv.invoice_id = ivl.invoice_id
    group by iv.invoice_id
    limit 10
'''
pd.read_sql(query, conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,...,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id,track_id,unit_price,quantity,num_purchases
0,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,...,NY,USA,10012-2612,15.84,16,1,1173,0.99,1,16
1,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,...,ON,Canada,K2P 1L7,9.90,26,2,2651,0.99,1,10
2,40,Dominique,Lefebvre,None,"8, Rue Hanovre",Paris,None,France,75002,+33 01 47 42 71 71,...,None,France,75002,1.98,28,3,2646,0.99,1,2
3,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,...,NY,USA,10012-2612,7.92,36,4,2514,0.99,1,8
4,27,Patrick,Gray,None,1033 N Park Ave,Tucson,AZ,USA,85719,+1 (520) 622-4200,...,AZ,USA,85719,16.83,53,5,2002,0.99,1,17
5,31,Martha,Silk,None,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,+1 (902) 450-0450,...,NS,Canada,B3S 1C5,1.98,55,6,30,0.99,1,2
6,49,Stanisław,Wójcik,None,Ordynacka 10,Warsaw,None,Poland,00-358,+48 22 828 37 39,...,None,Poland,00-358,10.89,66,7,42,0.99,1,11
7,59,Puja,Srivastava,None,"3,Raj Bhavan Road",Bangalore,None,India,560001,+91 080 22289999,...,None,India,560001,9.90,76,8,1560,0.99,1,10
8,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,...,NY,USA,10012-2612,8.91,85,9,196,0.99,1,9
9,31,Martha,Silk,None,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,+1 (902) 450-0450,...,NS,Canada,B3S 1C5,1.98,87,10,3290,0.99,1,2


In [86]:
query = '''
    select customer_name,
           units_purchased,
           money_spent

    from (select 
            c.first_name || " " || c.last_name customer_name,
            count(*) units_purchased,
            iv.total money_spent
            

        from customer c
        inner join invoice iv on c.customer_id = iv.customer_id
        inner join invoice_line ivl on iv.invoice_id = ivl.invoice_id

        group by iv.invoice_id)
    
    order by customer_name
    
'''
pd.read_sql(query, conn)

# one line = one purchasement done by a customer.
# Therefore,

# number_of_purchases = count(*) group by customer_name
# total_spent = sum(money_spent) group by customer_name

,customer_name,units_purchased,money_spent
0,Aaron Mitchell,20,19.80
1,Aaron Mitchell,11,10.89
2,Aaron Mitchell,4,3.96
3,Aaron Mitchell,2,1.98
4,Aaron Mitchell,7,6.93
5,Aaron Mitchell,7,6.93
6,Aaron Mitchell,6,5.94
7,Aaron Mitchell,14,13.86
8,Alexandre Rocha,3,2.97
9,Alexandre Rocha,3,2.97


In [87]:
query = '''
    select DISTINCT(customer_name) customer_name,
           COUNT(*) number_of_purchases,
           SUM(money_spent) total_spent,
           CASE
                WHEN SUM(money_spent) < 40 THEN 'small spender'
                ELSE
                    (CASE
                        WHEN SUM(money_spent) > 100 THEN 'big spender'
                        ELSE 'regular'
                        END)
                END
                AS customer_category

    from (select 
            c.first_name || " " || c.last_name customer_name,
            count(*) units_purchased,
            iv.total money_spent
            

        from customer c
        inner join invoice iv on c.customer_id = iv.customer_id
        inner join invoice_line ivl on iv.invoice_id = ivl.invoice_id

        group by iv.invoice_id)
    
    group by customer_name
    order by customer_name
    
'''
pd.read_sql(query, conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular


## Simple Approach (Dataquest Answer)
* We don't need to use 'invoice_line' table actually.
* Usage of `CASE` could be simple as below.

### Understanding the relationship among data sets is important;
* We can find the simplest solution to solve the problem with the smallest effort & resources.

In [81]:
q = '''
    SELECT
       c.first_name || " " || c.last_name customer_name,
       COUNT(i.invoice_id) number_of_purchases,
       SUM(i.total) total_spent,
       CASE
           WHEN sum(i.total) < 40 THEN 'small spender'
           WHEN sum(i.total) > 100 THEN 'big spender'
           ELSE 'regular'
           END
           AS customer_category
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 1 ORDER BY 1;
'''
pd.read_sql(q, conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
5,Daan Peeters,7,60.39,regular
6,Dan Miller,12,95.04,regular
7,Diego Gutiérrez,5,39.60,small spender
8,Dominique Lefebvre,9,72.27,regular
9,Eduardo Martins,12,60.39,regular


## Next Steps

In this mission, we learned how to work with databases with more than two tables, including how to:

* write queries that join 3 or more tables
* combine multiple joins with subqueries
* create recursive joins
* use `LIKE` to incorporate partial matches
* generate conditional columns using CASE

In the next mission, we'll continue to work with the Chinook database while we learn how to:

* write increasingly complex queries
* construct and format queries that are easy to read
* create temporary and permanent views using with and view
* use set operations to combine rows

In [88]:
conn.close()